In [3]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.base import clone as clone_estimator
import tlib
import concurrent.futures

# Pseudo random state
random_state = 42

# Load data
X, y = tlib.load_tfw1()
X2, y2 = tlib.load_tfw2()

executor = concurrent.futures.ProcessPoolExecutor()

def predict_secret(estimator, X, y, X2, y2, i):
    from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
    
    rs = random_state + i if isinstance(random_state, int) else random_state
    X_shuffled, y_shuffled = shuffle(X, y, random_state=rs)
    clone = clone_estimator(estimator)
    clone.fit(X_shuffled, y_shuffled)
    y_pred = clone.predict(X2)
    y_proba = clone.predict_proba(X2)[:, 1]
    
    cm = confusion_matrix(y2, y_pred)
    tp = cm[1, 1]
    fp = cm[0, 1]
    tn = cm[0, 0]
    fn = cm[1, 0]
    
    return f1_score(y2, y_pred), roc_auc_score(y2, y_proba), tp, fp, tn, fn

def benchmark_classifier(name, estimator, X, y, X2, y2):
    from sklearn.model_selection import cross_val_score, cross_validate
    from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
    from sklearn.metrics import make_scorer, confusion_matrix, f1_score, precision_score
    
    def tn_scorer(y_true, y_pred):
        return confusion_matrix(y_true, y_pred)[0, 0]
    def fp_scorer(y_true, y_pred):
        return confusion_matrix(y_true, y_pred)[0, 1]
    def fn_scorer(y_true, y_pred):
        return confusion_matrix(y_true, y_pred)[1, 0]
    def tp_scorer(y_true, y_pred):
        return confusion_matrix(y_true, y_pred)[1, 1]

    scoring = {
        "f1": "f1",
        "accuracy": "accuracy",
        "precision": make_scorer(precision_score, zero_division=1),
        "recall": "recall",
        "tn": make_scorer(tn_scorer),
        "fp": make_scorer(fp_scorer),
        "fn": make_scorer(fn_scorer),
        "tp": make_scorer(tp_scorer),
        "roc_auc": "roc_auc",
    }

    print(f"> Running model {name}...")
    results = cross_validate(estimator, X, y, cv=StratifiedKFold(shuffle=True), scoring=scoring)
    duration = np.max(results["fit_time"])
    print(f"  Done in {duration}s")

    # Compute auroc score
    # See https://elitedatascience.com/imbalanced-classes (#3 Change Your Performance Metric)
    # See https://stats.stackexchange.com/questions/132777/what-does-auc-stand-for-and-what-is-it
#     prob_y = estimator.predict_proba(X_test)
#     prob_y = [p[1] for p in prob_y]
#     auroc = roc_auc_score(y_test, prob_y)

    # Create a list to store the future objects
    futures = []
    f1_secret = []
    roc_auc_secret = []
    secret_tp = []
    secret_fp = []
    secret_tn = []
    secret_fn = []

    # Submit the function for execution in parallel for each iteration
    print(f"  Evaluating classifier on test dataset...")
    for i in range(10):
        f1, auroc, tp, fp, tn, fn = predict_secret(estimator, X, y, X2, y2, i)
        f1_secret.append(f1)
        roc_auc_secret.append(auroc)
        secret_tp.append(tp)
        secret_fp.append(fp)
        secret_tn.append(tn)
        secret_fn.append(fn)

    return name, results, f1_secret, roc_auc_secret, secret_tp, secret_fp, secret_tn, secret_fn

def benchmark_classifiers(X, y, competitors):
    """
    Benchmarks the given classifiers by training them on the same train and test data sets
    """
    
    # Container for the table rows
    futures_ = []
    data = []

    # Submit the function for execution in parallel for each iteration
    for competitor in competitors:
        name, estimator = competitor
        name, results, f1_secret, roc_auc_secret, secret_tp, secret_fp, secret_tn, secret_fn = benchmark_classifier(name, estimator, X, y, X2, y2)

#         future = executor.submit(benchmark_classifier, name, estimator, X, y, X2, y2)
#         futures_.append(future)

#     # Iterate over the completed futures to get the results
#     for future in concurrent.futures.as_completed(futures_):
#         name, results, f1_secret = future.result()

        data.append([
            name,
            np.max(results["fit_time"]),
            np.mean(results["test_accuracy"]),
            np.mean(results["test_tp"]),
            np.mean(results["test_fp"]),
            np.mean(results["test_tn"]),
            np.mean(results["test_fn"]),
            np.mean(results["test_f1"]),
            np.std(results["test_f1"]),
            np.mean(results["test_roc_auc"]),
            np.std(results["test_roc_auc"]),
#             f1,
#             0,
#             np.mean(results["test_f1_2"]),
#             np.std(results["test_f1_2"]),
            np.mean(f1_secret),
            np.std(f1_secret),
            np.mean(roc_auc_secret),
            np.std(roc_auc_secret),
            np.mean(secret_tp),
            np.mean(secret_fp),
            np.mean(secret_tn),
            np.mean(secret_fn),
        ])
    
    return data

# Import models from sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from xgboost import XGBRFClassifier

# Run all models
rows = benchmark_classifiers(X, y, [
    ("GradientBoosting", GradientBoostingClassifier(random_state=random_state)),
    ("HistGradientBoosting", HistGradientBoostingClassifier(random_state=random_state)),
    ("DecisionTree", DecisionTreeClassifier(random_state=random_state)),
    ("AdaBoost", AdaBoostClassifier(random_state=random_state)),
    ("ExtraTrees", ExtraTreesClassifier(random_state=random_state)),
    ("RandomForest", RandomForestClassifier(random_state=random_state)),
    ("xgbDART", XGBClassifier(random_state=random_state, booster="dart")),
    ("xgbTree", XGBClassifier(random_state=random_state)),
    ("Dummy", DummyClassifier(random_state=random_state, strategy="most_frequent")),
])

# Print results table
table = pd.DataFrame(rows, columns=[
    "Algorithm",
    "Time",
    "Accuracy",
    "True Pos",
    "False Pos",
    "True Neg",
    "False Neg",
    "F1",
    "F1 SD",
    "AUROC",
    "AUROC SD",
    "F1 (secret data)",
    "F1 SD (secret data)",
    "AUROC (secret data)",
    "AUROC SD (secret data)",
    "TP (secret)",
    "FP (secret)",
    "TN (secret)",
    "FN (secret)",
])
tlib.print_df(table)

> Running model GradientBoosting...


KeyboardInterrupt: 

In [3]:
table.to_csv("fin_01_model_evaluation_random_state=42,more.csv")

In [4]:
import pandas as pd
import numpy as np
import tlib

df = pd.read_csv("fin_01_model_evaluation_random_state=None,more.csv")

df["score"] = 0.5 * (df["F1"] + df["AUROC"])
df["score_f1"] = 0.5 * (df["F1"] + df["F1 (secret data)"])
df["score_public"] = 0.5 * (df["F1 (secret data)"] + df["AUROC (secret data)"])
df["score_total"] = 0.25 * (df["F1"] + df["AUROC"] + df["F1 (secret data)"] + df["AUROC (secret data)"])

tlib.print_df(df)

df = df.drop(df.index[[9]])
print(np.mean(df["F1"]))

FileNotFoundError: [Errno 2] No such file or directory: 'fin_01_model_evaluation_random_state=None,more.csv'

In [4]:
from sklearn.model_selection import train_test_split
import tlib

X, y = tlib.load_tfw1()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import SklearnClassifier

# Step 1: Create or load your pre-trained GradientBoostingClassifier model
# For demonstration purposes, let's create a random GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = GradientBoostingClassifier()
model.fit(X_train.to_numpy(), y_train.to_numpy())

def generate_adversarial_examples(model, X, y, epsilon=0.01):
    from tqdm import trange
    import sys
    
    perturbed_X = np.copy(X)
    
    # Iterate over each sample
    for i in trange(len(X), file=sys.stdout):
        # Get the original prediction
        original_prediction = model.predict([X[i]])
        
        # Get the feature indices sorted by their importance
        feature_indices = np.argsort(model.feature_importances_)[::-1]
        
        # Iterate over each feature and perturb it
        for feature_index in feature_indices:
            perturbed_X[i][feature_index] += epsilon
            
            # Check if the perturbed sample is misclassified
            if model.predict([perturbed_X[i]]) != original_prediction:
                break  # Stop perturbing the features
            
            # If not misclassified, reset the perturbed feature
            perturbed_X[i][feature_index] -= epsilon
    
    return perturbed_X

# Example usage
# Assume you already have a trained GradientBoostingClassifier model
# X is the input features, and y is the corresponding labels

# Generate adversarial examples
adversarial_X = generate_adversarial_examples(model, X_test.to_numpy(), y_test.to_numpy(), epsilon=0.04)

from sklearn.metrics import f1_score

f1_test = f1_score(y_test.to_numpy(), model.predict(X_test.to_numpy()))
f1_adv = f1_score(y_test.to_numpy(), model.predict(adversarial_X))

# Print the predicted labels for the adversarial examples
print(f1_test)
print(f1_adv)

100%|██████████| 27914/27914 [00:17<00:00, 1583.03it/s]
0.957787481804949
0.018101179100710345
